**Добрый день! Благодарю за интересное тестовое задание. Ниже по ходу выполнения буду комментировать свои действия в формате markdown.**

# Загрузка библиотек и данных

Загружаю библиотеку pandas

In [1]:
import pandas as pd

Загружаю файл csv с данными через path для вашего удобства. 
В момент загрузки файла выяснилось, что он был неправильно закодирован и некорректно отображал данные. Пришлось открыть в экселе и перекодировать в utf-8.

In [2]:
path = 'C:/Users/Sviridov/Desktop/Тест/'
df = pd.read_csv(path + 'Данные Python.csv', sep = ';')
df.head()

,Договор,Приложение
0,"ДД1, ДД187, ДД891",1.687.7897
1,ДД2,2
2,ДД3,3
3,ДД4 ДД567,465 47
4,ДД5,5


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Договор     501 non-null    object
 1   Приложение  501 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


Т.к. строк немного, есть возможность взглянуть на все уникальные значения столбцов для понимания полной картины. Скрыл код, чтобы не перегружать файл.

In [4]:
#sorted(df['договор'].unique())

In [5]:
#sorted(df['приложение'].unique())

**Итог**: учитывая задачу, принял решение пойти путем формирования списков в столбцах, а затем объединить их по номерам элементов списка в одном столбце согласно заданным условиям.

# Предобработка данных

Для удобства изменю названия столбцов на нижний регистр

In [6]:
df = df.rename(columns={'Договор':'договор','Приложение':'приложение'})

Определив все имеющиеся разделители последовательно меняю их на запятую

In [7]:
df['договор'] = df['договор'].str.replace('\n', ',')
df['договор'] = df['договор'].str.replace('/', ',')
df['договор'] = df['договор'].str.replace('.', ',',  regex=True)
df['договор'] = df['договор'].str.replace(' ', ',')
df['договор'] = df['договор'].str.replace('\\', ',', regex=True)

Методом strip очищаю данные от образовавшихся запятых в начале и в конце значений 

In [8]:
df['договор'] = df['договор'].str.strip(',')

В некоторых значениях уже фигурировали запятые, поэтому заменяю дубли запятых на одиночную запятую. 
По какой-то причине команда ниже не срабатывает при загрузке с первого раза, поэтому следом продублировал её для работоспособности кода. Списал данное недоразумение на баг.

In [9]:
df['договор'] = df['договор'].str.replace(',,', ',')

In [10]:
df['договор'] = df['договор'].str.replace(',,', ',')

Проделываю аналогичные действия с вторым столбцом.

In [11]:
df['приложение'] = df['приложение'].str.replace('\n', ',')
df['приложение'] = df['приложение'].str.replace('.', ',', regex=True)
df['приложение'] = df['приложение'].str.replace(' ', ',')
df['приложение'] = df['приложение'].str.replace('/', ',')

In [12]:
df['приложение'] = df['приложение'].str.strip(',')

In [13]:
df['приложение'] = df['приложение'].str.replace(',,', ',', regex=True)

In [14]:
df.head()

,договор,приложение
0,"ДД1,ДД187,ДД891","1,687,7897"
1,ДД2,2
2,ДД3,3
3,"ДД4,ДД567","465,47"
4,ДД5,5


**Итог**: теперь все данные в столбцах разделены только одной запятой.

# Формирование списков

С помощью лямбда функций создадим два столбца, в которых построчно сформируем списки с элементами через разделитель 'запятая'

In [15]:
df['договор_list'] = df["договор"].apply(lambda row: list(row.split(',')))
df['приложение_list'] = df["приложение"].apply(lambda row: list(row.split(',')))
pd.options.display.max_rows = None

In [16]:
df.head()

,договор,приложение,договор_list,приложение_list
0,"ДД1,ДД187,ДД891","1,687,7897","[ДД1, ДД187, ДД891]","[1, 687, 7897]"
1,ДД2,2,[ДД2],[2]
2,ДД3,3,[ДД3],[3]
3,"ДД4,ДД567","465,47","[ДД4, ДД567]","[465, 47]"
4,ДД5,5,[ДД5],[5]


**Итог**: получили готовые списки

# Объеденение элементов списков из разных столбцов

Начнем с условия 'кол-во Договоров = кол-во Приложений'. Для данной задачи также воспользуемся лямбда функцией с построчным действием:
-  создаем столбец 'ключ';
-  условие выполняем с помощью len (в нашем случае показывает кол-во элементов в списке каждой строки);
-  соединяем элементы через f-строку переменными x, y методом zip, который соединяет элементы по принципу первый с первым, второй со вторым и так далее, что и необходимо по услови;
-  через else сформируем пустой список. В последующем будем использовать row['ключ'], чтобы не затереть уже полученные ключи по предыдущим условиям.

In [17]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{x}{y}" for x, y in zip(row['договор_list'], row['приложение_list'])]
        if len(row['договор_list']) == len(row['приложение_list']) 
        else [] 
    ),
    axis=1
)

Дальше аналогично прорабатываем условие'кол-во Договоров = 1, а кол-во Приложений > 1', но вместо метода zip вручную определяем алгоритм соединения.

In [18]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{row['договор_list'][0]}{x}" for x in row['приложение_list']] 
        if len(row['договор_list']) == 1 and len(row['приложение_list'])>1
        else row['ключ']
    ),
    axis=1
)

Аналогично 'кол-во Договоров > 1, а кол-во Приложений = 1'

In [19]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{x}{row['приложение_list'][0]}" for x in row['договор_list']] 
        if len(row['договор_list']) > 1 and len(row['приложение_list']) == 1  
        else row['ключ']
    ),
    axis=1
)

Посмотрим какие строки остались для последнего условия.

In [20]:
df[df['ключ'].str.len() == 0]

,договор,приложение,договор_list,приложение_list,ключ
42,"ДД43,ДД611","42,5849,420,232,9786,56,12,88","[ДД43, ДД611]","[42, 5849, 420, 232, 9786, 56, 12, 88]",[]
55,"ДД56,ДД763,ДД195","55,98,75,1,888,567","[ДД56, ДД763, ДД195]","[55, 98, 75, 1, 888, 567]",[]
487,"ДД488,ДД2435","487,4343,6875,446","[ДД488, ДД2435]","[487, 4343, 6875, 446]",[]


К сожалению я не смог выполнить задачу одной командой для условия: 
if len(row['Договор_list']) > 1 and len(row['Приложение_list']) > 1 and len(row['Договор_list']) != len(row['Приложение_list']). Полагаю что нужно комбинировать с методами библиотеки numpy.
Поэтому разбил на подзадачи c возможными вариациями ниже.

'кол-во Договоров == 2, кол-во Приложений == 8 ;

In [21]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{row['договор_list'][0]}{x}" for x in row['приложение_list'][:4]]+
        [f"{row['договор_list'][1]}{x}" for x in row['приложение_list'][4:]]
        if len(row['договор_list']) == 2 and len(row['приложение_list']) == 8
        else row['ключ']
    ),
    axis=1
)

'кол-во Договоров == 3, кол-во Приложений == 6 ;

In [22]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{row['договор_list'][0]}{x}" for x in row['приложение_list'][:2]]+
        [f"{row['договор_list'][1]}{x}" for x in row['приложение_list'][2:4]]+
        [f"{row['договор_list'][2]}{x}" for x in row['приложение_list'][4:]]
        if len(row['договор_list']) == 3 and len(row['приложение_list']) == 6
        else row['ключ']
    ),
    axis=1
)

'кол-во Договоров == 2, кол-во Приложений == 4 ;

In [23]:
df['ключ'] = df.apply(
    lambda row: (
        [f"{row['договор_list'][0]}{x}" for x in row['приложение_list'][:2]]+
        [f"{row['договор_list'][1]}{x}" for x in row['приложение_list'][2:]]
        if len(row['договор_list']) == 2 and len(row['приложение_list']) == 4
        else row['ключ']
    ),
    axis=1
)

Проверка на заполненность всех строк в столбце 'ключ'

In [24]:
df[df['ключ'].str.len() == 0]

,договор,приложение,договор_list,приложение_list,ключ


**Итог**: получили столбец 'ключ' c готовыми ключами по заданным условиям

# Формирование финального общего списка ключей

Воспользумся методами sum и tolist. С помощью tolist сформируем список списков из столбца 'ключ'. А с помощью второго аргумента [] в sum произойдет соединение всех элементов списков в один.

In [25]:
список_ключей = sum(df['ключ'].tolist(), [])
список_ключей

['ДД11',
 'ДД187687',
 'ДД8917897',
 'ДД22',
 'ДД33',
 'ДД4465',
 'ДД56747',
 'ДД55',
 'ДД66',
 'ДД77',
 'ДД88',
 'ДД74198',
 'ДД8998',
 'ДД99',
 'ДД1010',
 'ДД1111',
 'ДД1212',
 'ДД1313',
 'ДД14145',
 'ДД1476',
 'ДД145',
 'ДД1498',
 'ДД1515',
 'ДД1616',
 'ДД1717',
 'ДД1818',
 'ДД1919',
 'ДД2020',
 'ДД2121',
 'ДД2222',
 'ДД2323',
 'ДД2424',
 'ДД2525',
 'ДД77765',
 'ДД2626',
 'ДД2727',
 'ДД2828',
 'ДД2929',
 'ДД3030',
 'ДД3131',
 'ДД3232',
 'ДД3333',
 'ДД3434',
 'ДД3475',
 'ДД34460',
 'ДД3456768',
 'ДД34568',
 'ДД343567',
 'ДД3468776',
 'ДД344657',
 'ДД3535',
 'ДД3636',
 'ДД3737',
 'ДД3838',
 'ДД3939',
 'ДД401',
 'ДД5432',
 'ДД8983',
 'ДД9994',
 'ДД4140',
 'ДД4241',
 'ДД4342',
 'ДД435849',
 'ДД43420',
 'ДД43232',
 'ДД6119786',
 'ДД61156',
 'ДД61112',
 'ДД61188',
 'ДД4443',
 'ДД4544',
 'ДД4645',
 'ДД4746',
 'ДД4847',
 'ДД4948',
 'ДД5049',
 'ДД5150',
 'ДД5251',
 'ДД5352',
 'ДД345567',
 'ДД5453',
 'ДД123453',
 'ДД5554',
 'ДД5655',
 'ДД5698',
 'ДД76375',
 'ДД7631',
 'ДД195888',
 'ДД195567',

In [26]:
len(список_ключей)

567

**Итог**: получили финальный список ключей с количеством 567.